In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams,PointStruct
import pandas as pd
import json

In [5]:


# # Load the embeddings from the JSON file
# with open('embeddings.json', 'r') as file:
#     embeddings = json.load(file)

# # Now 'embeddings' contains the list of loaded embeddings
# print(embeddings[0])


In [6]:
df =pd.read_csv('corpus.csv')
data_list_of_dicts =df.to_dict(orient='records')

data_list_of_dicts[0]

{'text': 'Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.',
 'cid': 0}

In [7]:

client = QdrantClient(path='qdrant_db')

collection_name = "legal_hackthon_2024_ver1"
if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=768, distance=Distance.COSINE),
    )

print(client.get_collections())
number_points = client.count(collection_name).count
print(number_points)
# client.close()

collections=[CollectionDescription(name='legal_hackthon_2024_ver1')]
0


In [17]:
for i in range(1,10):
    with open(f'embeddings_part_{i+1}.json', 'r') as file:
        embeddings = json.load(file)
    points=[]
    number_points = client.count(collection_name).count
    print(number_points)
    for idx, vec in enumerate(embeddings):
        curr=PointStruct(id = idx+number_points,vector=vec,payload=data_list_of_dicts[idx+number_points])
        points.append(curr)
    client.upsert(
        collection_name=collection_name,
        points=points,
        wait=True
    )
    print(f'insert embedding part {i+1} successfully!')

26160
insert embedding part 2 successfully!
52320
insert embedding part 3 successfully!
78480
insert embedding part 4 successfully!
104640
insert embedding part 5 successfully!
130800
insert embedding part 6 successfully!
156960
insert embedding part 7 successfully!
183120
insert embedding part 8 successfully!
209280
insert embedding part 9 successfully!
235440
insert embedding part 10 successfully!


In [11]:
apoint=client.retrieve(collection_name=collection_name,ids=[0])